In [47]:
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from tokenizers import ByteLevelBPETokenizer
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt

In [54]:
from dataset import encoding
from model_joint import make_model
from model_joint import LabelSmoothing
from model_joint import NoamOpt
from model_joint import data_gen
from belief_accuracy import joint_accuracy
from belief_accuracy import slot_accuracy
from action_belief_accuracy import obtain_TP_TN_FN_FP
import torch

In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [28]:
tokenizer = ByteLevelBPETokenizer('simpletod/vocab.json','simpletod/merges.txt')
tokenizer.add_special_tokens(["<PAD>","<|belief|>"," <|endofbelief|>","<|context|>"," <|user|> "," <|system|> ","<|endofcontext|>","<|action|>"," <|endofaction|>"])

9

In [29]:
pad_tok = tokenizer.encode("<PAD>").ids.pop()
print(pad_tok)

18157


In [8]:
def print_ele(elem_list):
  for ele in elem_list:
    if tokenizer.decode([ele],skip_special_tokens=False) == '<PAD>':
      return
    yield tokenizer.decode([ele],skip_special_tokens=False)

In [30]:
def run_epoch(data_iter, model, loss_compute):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    loss1 = []
    index = []
    ref1 = []
    cand1 = []
    #file1 = open("test_file2.txt", "a")
    for i, batch in enumerate(data_iter):
        reference = []
        candidate = []
        ref2 = []
        cand2 = []
        out_belief, out_action = model.forward_belief_action(batch.src.to(device), batch.trg1.to(device), 
                            batch.src_mask.to(device), batch.trg1_mask.to(device),batch.trg2.to(device),batch.trg2_mask.to(device))
        #out_action = model.forward_action(batch.src.to(device), batch.trg2.to(device), 
                            #batch.src_mask.to(device), batch.trg2_mask.to(device))
        loss = loss_compute(out_belief,out_action, batch.trg1_y.to(device),batch.trg2_y.to(device), batch.ntokens1.to(device),batch.ntokens2.to(device))
        loss1.append(loss)
        index.append(i)
        total_loss += loss
        total_tokens += (batch.ntokens1+batch.ntokens2)
        tokens += (batch.ntokens1+batch.ntokens2)
        #file1.write("Batch No. - %d \n" % (i+1))
        #max , idx = torch.max(model.generator(out),dim=2)
        #pred2 = idx.tolist()
        #for d in range(len(batch.src.tolist())):
                #file1.write(''.join(list(print_ele(batch.src.tolist()[d]))) + "\n")#tokenizer.decode(batch.src.tolist()[d],skip_special_tokens=False) + "\n")
                #file1.write("<|belief|>" + tokenizer.decode(pred2[d],skip_special_tokens=True) + " <|endofbelief|>" + "\n")
                #file1.write("<|belief|>" + tokenizer.decode(batch.trg_y.tolist()[d],skip_special_tokens=True) + " <|endofbelief|>" + "\n")
                #ref1.append(tokenizer.decode(pred2[d],skip_special_tokens=True).split(','))
                #cand1.append(tokenizer.decode(batch.trg_y.tolist()[d],skip_special_tokens=True).split(','))
                #ref2.append(tokenizer.decode(pred2[d],skip_special_tokens=True).split(','))
                #cand2.append(tokenizer.decode(batch.trg_y.tolist()[d],skip_special_tokens=True).split(','))
        #file1.write("Joint Accuracy - %f \n" % joint_accuracy(cand2,ref2,d_c = False,type2_c = False))
        #file1.write("Slot Accuracy - %f \n" % slot_accuracy(cand2,ref2))
        if i % 200 == 1:
            elapsed = time.time() - start
            '''max , idx = torch.max(model.generator(out),dim=2)
            a = idx.tolist()
            b = batch.trg_y.tolist()
            print(idx.size())
            print(batch.trg_y.size())
            for id in a:
                reference.append(tokenizer.decode(id,skip_special_tokens=True).split(','))
            for p in batch.trg_y.tolist():
                candidate.append(tokenizer.decode(p,skip_special_tokens=True).split(',')) 
            print(reference)
            print(candidate)
            accuracy = joint_accuracy(candidate,reference,d_c = False,type2_c = False)
            slt_accuracy = slot_accuracy(candidate,reference)'''
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f " %
                    (i, loss / (batch.ntokens1+batch.ntokens2), tokens / elapsed))
            start = time.time()
            tokens = 0
    #file1.write("Overall Joint Accuracy - %f \n" % joint_accuracy(cand1,ref1,d_c = False,type2_c = False))
    #file1.write("Overall Slot Accuracy - %f \n" % slot_accuracy(cand1,ref1))
    #file1.close()
    return total_loss / total_tokens#,accuracy,slt_accuracy

**Loss Compute Cell**

In [9]:
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator,generator1, criterion, opt=None):
        self.generator = generator
        self.generator1 = generator1
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x1,x2, y,z, norm1,norm2):
        x1 = self.generator(x1)
        x2 = self.generator1(x2)
        loss1 = self.criterion(x1.contiguous().view(-1, x1.size(-1)), 
                              y.contiguous().view(-1)) / norm1
        loss2 = self.criterion(x2.contiguous().view(-1, x2.size(-1)), 
                              z.contiguous().view(-1)) / norm2
        loss = loss1 + loss2
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.item() * norm1

In [ ]:
V = 30000
model = torch.load('simpletod/dialog_NLP.pt')

**Initial one**

In [23]:
criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)


s7,t7,a7 = encoding('simpletod/Processed_data/simptod/context_test.txt','simpletod/Processed_data/simptod/belief_test.txt','simpletod/Processed_data/simptod/action_test.txt',400)
l2 = run_epoch(data_gen(V, 10, 737,s7,t7,a7), model.to(device), 
                    SimpleLossCompute(model.generator,model.generator1, criterion, None))
print(l2)
#print(a3)
#print(sa3)

C:\Users\alipu\anaconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch Step: 1 Loss: 11.673458 Tokens per Sec: 1564.010620 
Epoch Step: 201 Loss: 11.669713 Tokens per Sec: 2995.048828 
Epoch Step: 401 Loss: 11.673076 Tokens per Sec: 8254.501953 
Epoch Step: 601 Loss: 11.660394 Tokens per Sec: 10573.483398 
tensor(11.6703, device='cuda:0')


**Final one**

### For Belief prediction

In [11]:
start = tokenizer.encode("<|belief|>").ids.pop()

In [12]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    words = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out_belief = model.decode_belief(memory, src_mask, 
                           Variable(words), 
                           Variable(subsequent_mask(words.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out_belief[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()
        words = torch.cat([words, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    if words.dim() == 2:
        words = words.squeeze(0)
        words = words.tolist()
        
    sen_idx = [w for w in words]
    sentence = tokenizer.decode(sen_idx,skip_special_tokens=True)#' '.join([rev_word_map[sen_idx[k]] for k in range(len(sen_idx))])
    
    return sentence

In [ ]:
s4,t4,a4= encoding('simpletod/Processed_data/simptod/context_test.txt','simpletod/Processed_data/simptod/belief_test.txt','simpletod/Processed_data/simptod/action_test.txt',400)
file1 = open("test_file_belief.txt", "a")
for i in range(7370):
    src = Variable(torch.unsqueeze(s4[i,:400],0))
    src_mask = Variable(torch.ones(1, 1, 400) )
    s = greedy_decode(model.to(device), src.to(device), src_mask.to(device), 60, start)
    file1.write(''.join(list(print_ele([w for w in s4[i][:400]]))) + "\n")#tokenizer.decode(batch.src.tolist()[d],skip_special_tokens=False) + "\n")
    file1.write("<|belief|>" + s + " <|endofbelief|>" + "\n")
    file1.write("<|belief|>" + tokenizer.decode([w for w in t4[i][:400]],skip_special_tokens=True) + " <|endofbelief|>" + "\n")
file1.close()

### For Action prediction

In [14]:
start = tokenizer.encode("<|action|>").ids.pop()

In [15]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    words = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out_action = model.decode_action(memory, src_mask, 
                           Variable(words), 
                           Variable(subsequent_mask(words.size(1))
                                    .type_as(src.data)))
        prob = model.generator1(out_action[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()
        words = torch.cat([words, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    if words.dim() == 2:
        words = words.squeeze(0)
        words = words.tolist()
        
    sen_idx = [w for w in words]
    sentence = tokenizer.decode(sen_idx,skip_special_tokens=True)#' '.join([rev_word_map[sen_idx[k]] for k in range(len(sen_idx))])
    
    return sentence

In [ ]:
file1 = open("test_file_action.txt", "a")
for i in range(7370):
    src = Variable(torch.unsqueeze(s4[i,:400],0))
    src_mask = Variable(torch.ones(1, 1, 400) )
    s = greedy_decode(model.to(device), src.to(device), src_mask.to(device), 60, start)
    file1.write(''.join(list(print_ele([w for w in s4[i][:400]]))) + "\n")#tokenizer.decode(batch.src.tolist()[d],skip_special_tokens=False) + "\n")
    file1.write("<|action|>" + s + " <|endofaction|>" + "\n")
    file1.write("<|action|>" + tokenizer.decode([w for w in a4[i][:400]],skip_special_tokens=True) + " <|endofaction|>" + "\n")
file1.close()

### Joint Accuracy and Slot Accuracy

In [91]:
f=open('test_file_belief.txt','r',errors = 'ignore')
raw=f.read()
#raw=raw.lower()
output = raw.split("\n")

In [92]:
dict = {'target_turn_belief':[] ,'generated_turn_belief':[] ,'model_context':[] }
#dict = dict.fromkeys(['target_turn_belief','generated_turn_belief','model_context'])
for i in range(0,len(output)-3,3):
    dict['target_turn_belief'].append((' '.join(output[i+2].split()[1:-1]).split(',')))
    dict['generated_turn_belief'].append((' '.join(output[i+1].split()[1:-1]).split(',')))
    dict['model_context'].append((' '.join(output[i].split()[1:-1]).split(',')))

In [93]:
dict1 = {i:{'target_turn_belief':dict['target_turn_belief'][i:i+10],'generated_turn_belief':dict['generated_turn_belief'][i:i+10],'model_context':dict['model_context'][i:i+10]} for i in range(0,len(dict['target_turn_belief']),10)}

In [94]:
import json
with open("eval_file_belief.json", "w") as outfile:  
    json.dump(dict1, outfile)

In [95]:
!python simpletod/compute_joint_acc.py --eval_file eval_file_belief.json --type2_cleaning 

joint accuracy: 0.15082382762991128


In [96]:
!python simpletod/compute_joint_acc.py --eval_file eval_file_belief.json 

joint accuracy: 0.13688212927756654


In [97]:
tgt_belief = []
pred_belief = []
for i in range(0,len(output)-3,3):
    tgt_belief.append((' '.join(output[i+2].split()[1:-1]).split(',')))
    pred_belief.append((' '.join(output[i+1].split()[1:-1]).split(',')))

In [98]:
slot_accuracy = slot_accuracy(tgt_belief,pred_belief)

In [99]:
print(slot_accuracy)

0.42775665399239543


### Action accuracy

In [100]:
f=open('test_file_action.txt','r',errors = 'ignore')
raw=f.read()
#raw=raw.lower()
output = raw.split("\n")

In [101]:
tgt_action = []
pred_action = []
for i in range(0,len(output)-3,3):
    tgt_action.append(''.join((' '.join(output[i+2].split()[1:-1]).replace(',',''))).split())
    pred_action.append(''.join((' '.join(output[i+1].split()[1:-1]).replace(',',''))).split())

### Modifications are done in this cell

In [102]:
pred =[]
tgt = []
for i in range(len(pred_action)):
    #if len(tgt_action[i])==len(pred_action[i]):
        for j in range(0,len(tgt_action[i]),3):
            pred.append(' '.join(pred_action[i][j:j+3]))
            tgt.append(' '.join(tgt_action[i][j:j+3]))  

In [104]:
domains = ['restaurant', 'hotel', 'attraction', 'train', 'taxi', 'hospital', 'police', 'bus', 'booking', 'general']
functions = ['inform', 'request', 'recommend', 'book', 'select', 'sorry', 'none','reqmore']
arguments = ['pricerange', 'id', 'addr', 'post', 'type','', 'food', 'phone', 'name', 'area', 'choice', 
             'price', 'time', 'ref',  'parking', 'stars', 'internet', 'day', 'arrive', 'depart',
             'dest', 'leave','greet', 'duration', 'trainid','ticket', 'people', 'department', 'stay','car','bye','fee','nobook','nooffer','offerbook','welcome','offerbooked']
used_levels = domains + functions + arguments
keys = range(len(used_levels))
act_len = len(used_levels)
#pre1 = [0] * act_len
#pre2 = [0] * act_len

dicts = {}
for p in keys:
    dicts[p] = used_levels[p]
dict = {v: k for k, v in dicts.items()}

all_pred = []
all_label = []

for i in pred:
    pre1 = [0] * act_len
    for w in i.split(' '):
        pre1[dict[w]] = 1
    all_pred.append(pre1)
for j in tgt:
    pre2 = [0] * act_len
    for w in j.split(' '):
        pre2[dict[w]] = 1
    all_label.append(pre2)

In [105]:
all_pred = torch.Tensor(all_pred)
all_label = torch.Tensor(all_label)

TP, TN, FN, FP = 0 ,0,0,0

In [106]:
TP, TN, FN, FP = obtain_TP_TN_FN_FP(all_pred, all_label, TP, TN, FN, FP)
precision = TP / (TP + FP + 0.001)
recall = TP / (TP + FN + 0.001)
F1 = 2 * precision * recall / (precision + recall + 0.001)

In [107]:
print(TP)

24776


In [108]:
print(precision)

0.514227589418493


In [109]:
print(recall)

0.43211942876867193


In [110]:
print(F1)

0.46911581063890434
